In [96]:
import io
import json

from pathlib import Path
from urllib.parse import urlparse, parse_qs

import pandas as pd
import numpy as np

import requests
import clipboard

from PIL import Image
from jinja2 import Template

In [49]:
module_number = 1
module_name = 'Introduction'
module_folder = Path('..') / '01-intro'

## Prepare `meta.json` info

In [10]:
text = """
module	module_name	unit	title	full_title	youtube
1	Introduction	1	Introduction	MLOps Zoomcamp 1.1 - Introduction	https://www.youtube.com/watch?v=s0uaFZSzwfI
1	Introduction	2	Environment preparation	MLOps Zoomcamp 1.2 - Environment preparation	
1	Introduction	3	(Optional) Training a ride duration prediction model	MLOps Zoomcamp 1.3 - (Optional) Training a ride duration prediction model	https://www.youtube.com/watch?v=iRunifGSHFc
1	Introduction	4	Course overview	MLOps Zoomcamp 1.4 - Course overview	https://www.youtube.com/watch?v=teP9KWkP6SM
1	Introduction	5	MLOps maturity model	MLOps Zoomcamp 1.5 - MLOps maturity model	https://www.youtube.com/watch?v=XwTH8BDGzYk
1	Introduction	6	Homework	MLOps Zoomcamp 1.6 - Homework	NA
""".strip()

In [11]:
df = pd.read_csv(io.StringIO(text), delimiter='\t')

In [30]:
df['youtube'] = df['youtube'].fillna('')

In [33]:
units = df[['unit', 'title', 'youtube']].to_dict(orient='records')
units

[{'unit': 1,
  'title': 'Introduction',
  'youtube': 'https://www.youtube.com/watch?v=s0uaFZSzwfI'},
 {'unit': 2, 'title': 'Environment preparation', 'youtube': ''},
 {'unit': 3,
  'title': '(Optional) Training a ride duration prediction model',
  'youtube': 'https://www.youtube.com/watch?v=iRunifGSHFc'},
 {'unit': 4,
  'title': 'Course overview',
  'youtube': 'https://www.youtube.com/watch?v=teP9KWkP6SM'},
 {'unit': 5,
  'title': 'MLOps maturity model',
  'youtube': 'https://www.youtube.com/watch?v=XwTH8BDGzYk'},
 {'unit': 6, 'title': 'Homework', 'youtube': ''}]

In [62]:
meta = {
    'module_info': {
        'module_number': module_number,
        'module_name': module_name
    },
    'units': units
}

In [63]:
meta

{'module_info': {'module_number': 1, 'module_name': 'Introduction'},
 'units': [{'unit': 1,
   'title': 'Introduction',
   'youtube': 'https://www.youtube.com/watch?v=s0uaFZSzwfI'},
  {'unit': 2, 'title': 'Environment preparation', 'youtube': ''},
  {'unit': 3,
   'title': '(Optional) Training a ride duration prediction model',
   'youtube': 'https://www.youtube.com/watch?v=iRunifGSHFc'},
  {'unit': 4,
   'title': 'Course overview',
   'youtube': 'https://www.youtube.com/watch?v=teP9KWkP6SM'},
  {'unit': 5,
   'title': 'MLOps maturity model',
   'youtube': 'https://www.youtube.com/watch?v=XwTH8BDGzYk'},
  {'unit': 6, 'title': 'Homework', 'youtube': ''}]}

In [64]:
module_folder.mkdir(parents=True, exist_ok=True)
meta_json_file = module_folder / 'meta.json'

with open(meta_json_file, 'wt') as f_out:
    json.dump(meta, f_out, indent=2)

In [65]:
!head {meta_json_file}

{
  "module_info": {
    "module_number": 1,
    "module_name": "Introduction"
  },
  "units": [
    {
      "unit": 1,
      "title": "Introduction",
      "youtube": "https://www.youtube.com/watch?v=s0uaFZSzwfI"


## Generate page

In [66]:
meta_json_file = module_folder / 'meta.json'

In [108]:
with meta_json_file.open('rt') as f_in:
    meta = json.load(f_in)

In [68]:
module_info = meta['module_info']
units = meta['units']

In [142]:
module_info

{'module_number': 1, 'module_name': 'Introduction'}

In [89]:
images_folder = module_folder / 'images'
images_folder.mkdir(parents=True, exist_ok=True)

In [90]:
overwrite = True

In [138]:
template_string = """
## {{ module_number }}.{{ unit }} {{ title }}

{% if youtube %}<a href="{{ youtube }}">
  <img src="{{ thumbnail }}">
</a>{% endif %}{% if not youtube %}COMING SOON{% endif %}


""".lstrip()

template = Template(template_string)

In [139]:
def download_thumbnail(video, module, unit, folder):
    if type(unit) in [int, np.int64]:
        thumbnail_file = f'thumbnail-{module}-{unit:02d}.jpg'
    else:
        thumbnail_file = f'thumbnail-{module}-{unit}.jpg'

    thumbnail_file = folder / thumbnail_file

    if thumbnail_file.exists():
        print(f'{thumbnail_file} exists')
        return thumbnail_file

    video_id = parse_qs(urlparse(video).query)['v'][0]
    print(f'processing video {video_id}...')
    thumbnail_url = f'https://img.youtube.com/vi/{video_id}/0.jpg'

    response = requests.get(thumbnail_url)
    thumbnail = Image.open(io.BytesIO(response.content))
    w_img, h_img = thumbnail.size

    play = Image.open(Path('../images/play.png'))
    w_play, h_play = play.size
    
    x0 = w_img // 2 - w_play // 2
    y0 = h_img // 2 - h_play // 2

    thumbnail.paste(play, (x0, y0), play)
    thumbnail.save(thumbnail_file, quality=90)

    print('saved to', thumbnail_file)

    return thumbnail_file

In [140]:
module_number = module_info['module_number']

parts = []

for unit in units:
    youtube = unit['youtube']
    unit_number = unit['unit']

    params = {**module_info, **unit}

    if len(youtube) and youtube.startswith('https'):
        thumbnail = download_thumbnail(youtube, module_number, unit_number, images_folder)
        thumbnail_path = '/'.join(thumbnail.parts[2:])
        params['youtube'] = params['youtube'] + '&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK'
        params['thumbnail'] = thumbnail_path
        
    template_string = template.render(params)
    print(template_string)
    parts.append(template_string)

..\01-intro\images\thumbnail-1-01.jpg exists
## 1.1 Introduction

<a href="https://www.youtube.com/watch?v=s0uaFZSzwfI&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK">
  <img src="images/thumbnail-1-01.jpg">
</a>

### Notes

Add notes (PRs are welcome)


## 1.2 Environment preparation

<a href="&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK">
  <img src="">
</a>

### Notes

Add notes (PRs are welcome)


..\01-intro\images\thumbnail-1-03.jpg exists
## 1.3 (Optional) Training a ride duration prediction model

<a href="https://www.youtube.com/watch?v=iRunifGSHFc&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK">
  <img src="images/thumbnail-1-03.jpg">
</a>

### Notes

Add notes (PRs are welcome)


..\01-intro\images\thumbnail-1-04.jpg exists
## 1.4 Course overview

<a href="https://www.youtube.com/watch?v=teP9KWkP6SM&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK">
  <img src="images/thumbnail-1-04.jpg">
</a>

### Notes

Add notes (PRs are welcome)


..\01-intro\images\thumbnail-1-05.jpg exists
## 1.5 MLOps maturity 

In [148]:
prefix = f"""
# {module_info['module_number']}. {module_info['module_name']} 
""".strip()


final_result = '\n\n'.join([prefix] + parts)

In [152]:
clipboard.copy(final_result)